In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [5]:
data = requests.get(standings_url)

In [6]:
import mysql.connector

def openConnection():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="1998.Ronc@",
        database="futebol_data"
    )
    return conn

def closeConnection(conn):
    conn.close()

In [7]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
rows = standings_table.find_all('tr')

In [20]:
#Inserindo as informações a primeira vez no banco
conn = openConnection()
cursor = conn.cursor()
for i, row in enumerate(rows[1:], start=1):  # Ignora o cabeçalho
    columns = row.find_all('td')
    column_data = [col.text for col in columns]
    column_data.append(i)
    sql = """
    INSERT INTO team (team_name, matches_played, win, loss, draw, goals_for, goals_against, goals_difference, points, position)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s ,%s)
    """
    team_name = column_data[0].strip()
    # Modifique os índices de column_data conforme as colunas da tabela
    data_to_insert = (
            team_name,  # Nome do time
            int(column_data[1]),  # Jogos disputados
            int(column_data[2]),  # Vitórias
            int(column_data[3]),  # Derrotas
            int(column_data[4]),  # Empates
            int(column_data[5]),  # gols marcados
            int(column_data[6]),  # gols sofridos
            int(column_data[7]),  # saldo de gols
            int(column_data[8]),  # pontos
            int(column_data[19])  # Posição do time
        )
    cursor.execute(sql, data_to_insert)
    print(column_data)
# Commit das alterações
conn.commit()
# Fechando a conexão
cursor.close()
closeConnection(conn)

[' Liverpool', '8', '7', '0', '1', '15', '3', '+12', '21', '2.63', '15.7', '6.1', '+9.6', '+1.19', 'L W W W W', '60,246', 'Mohamed Salah, Luis Díaz - 5', 'Alisson', '', 1]
[' Manchester City', '8', '6', '2', '0', '19', '9', '+10', '20', '2.50', '15.4', '8.5', '+6.9', '+0.86', 'W D D W W', '52,790', 'Erling Haaland - 10', 'Ederson', '', 2]
[' Arsenal', '8', '5', '2', '1', '15', '8', '+7', '17', '2.13', '13.5', '9.1', '+4.4', '+0.56', 'W D W W L', '60,304', 'Kai Havertz - 4', 'David Raya', '', 3]
[' Aston Villa', '8', '5', '2', '1', '15', '10', '+5', '17', '2.13', '11.4', '8.5', '+2.9', '+0.36', 'W W D D W', '41,455', 'Ollie Watkins - 5', 'Emiliano Martínez', '', 4]
[' Brighton', '8', '4', '3', '1', '14', '10', '+4', '15', '1.88', '11.8', '13.3', '-1.4', '-0.18', 'D D L W W', '34,010', 'Danny Welbeck - 5', 'Bart Verbruggen', '', 5]
[' Chelsea', '8', '4', '2', '2', '17', '10', '+7', '14', '1.75', '15.4', '9.7', '+5.6', '+0.70', 'W W W D L', '39,528', 'Cole Palmer - 6', 'Robert Sánchez', '

In [9]:
#Código para atualizar as informações dos times. OBS: SÓ UTILIZAR SE JÁ TIVER INSERIDO OS TIMES DENTRO DO BANCO.
conn = openConnection()
cursor = conn.cursor()
for i, row in enumerate(rows[1:], start=1):  # Ignora o cabeçalho
    columns = row.find_all('td')
    column_data = [col.text for col in columns]
    column_data.append(i)
    
    # SQL para atualizar os dados do time
    sql_update = """
    UPDATE team 
    SET matches_played = %s, win = %s, loss = %s, draw = %s, 
        goals_for = %s, goals_against = %s, goals_difference = %s, 
        points = %s, position = %s
    WHERE team_name = %s
    """
    team_name = column_data[0].strip()
    
    # Modifique os índices de column_data conforme as colunas da tabela
    data_to_update = (
        int(column_data[1]),  # Jogos disputados
        int(column_data[2]),  # Vitórias
        int(column_data[3]),  # Derrotas
        int(column_data[4]),  # Empates
        int(column_data[5]),  # Gols marcados
        int(column_data[6]),  # Gols sofridos
        int(column_data[7]),  # Saldo de gols
        int(column_data[8]),  # Pontos
        int(column_data[19]),  # Posição
        team_name  # Nome do time (critério para atualização)
    )
    
    cursor.execute(sql_update, data_to_update)

# Commit das alterações
conn.commit()

# Fechando a conexão
cursor.close()
closeConnection(conn)

In [8]:
#tornando os links em formato completo
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/

In [9]:
#Função para traduzir sigla de nacionalidade
def translate_country(country):
    country_dic = {
        'NED': 'Holanda',
        'EGY': 'Egito',
        'ENG': 'Inglaterra',
        'FRA': 'França',
        'HUN': 'Hungria',
        'SCO': 'Escocia',
        'ARG': 'Argentina',
        'BRA': 'Brasil',
        'COL': 'Colombia',
        'IRL': 'Irlanda',
        'URU': 'Uruguai',
        'GRE': 'Grecia',
        'ITA': 'Italia',
        'CZE': 'Republica tcheca',
        'JPN': 'Japao',
        'POR': 'Portugal',
        'NIR': 'Irlanda do norte'
    }
    return country_dic.get(country, 'Não identificado')

In [10]:
#Função para traduzir sigla de posição
def translate_pos(pos):
    pos_dic = {
        'GK': 'Goleiro',
        'LF': 'Lateral esquerdo',
        'DF': 'Zagueiro',
        'RB': 'Lateral direito',
        'FB': 'Lateral',
        'CB': 'Zagueiro central',
        'DM': 'Volante',
        'CM': 'Meio-campista Central',
        'LM': 'Meio-campista Esquerdo',
        'RM': 'Meio-campista Direito',
        'WM': 'Meio-campista Aberto',
        'MF': 'Meio campo',
        'AM':  'Meio-campista Ofensivo',
        'LW': 'Ponta Esquerdo',
        'RW': 'Ponta Direito',
        'FW': 'Atacante',
    }
    return pos_dic.get(pos, 'Posição inválida')

In [11]:
# Função para garantir a conversão para inteiro, tratando erros
def safe_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return 0 

In [42]:
#Função para tratar nome do time
def fetch_team_name(team_name):
    team_dic = {
        'Liverpool': 'Liverpool',
        'Manchester City': 'Manchester City',
        'Arsenal': 'Arsenal',
        'Aston Villa': 'Aston Villa',
        'Brighton and Hove Albion': 'Brighton',
        'Brighton': 'Brighton',
        'Chelsea': 'Chelsea',
        'Tottenham Hotspur': 'Tottenham',
        'Tottenham': 'Tottenham',
        'Newcastle United': 'Newcastle Utd',
        'Newcastle Utd': 'Newcastle Utd',
        'Fulham': 'Fulham',
        'Bournemouth': 'Bournemouth',
        'Manchester United': 'Manchester Utd',
        'Manchester Utd': 'Manchester Utd',
        "Nottingham Forest": "Nott'ham Forest",
        "Nott'ham Forest": "Nott'ham Forest",
        'Brentford': 'Brentford',
        'Leicester City': 'Leicester City',
        'West Ham United': 'West Ham',
        'West Ham': 'West Ham',
        'Everton': 'Everton',
        'Ipswich Town': 'Ipswich Town',
        'Crystal Palace': 'Crystal Palace',
        'Southampton': 'Southampton',
        'Wolverhampton Wanderers': 'Wolves',
        'Wolves': 'Wolves'
    }
    return team_dic.get(team_name, 'Time inválido')



In [13]:
#Extração de informação dos players dos times pela primeira vez
conn = openConnection()
cursor = conn.cursor()
for team_url in team_urls:
    data = requests.get(team_url)
    players = pd.read_html(data.text, match="Standard Stats")[0]
    players.columns = players.columns.droplevel(0)
    players.columns = players.columns.get_level_values(-1) 
    players = players[:-2]
    players.columns = [f"{col}_{i}" if players.columns.duplicated()[i] else col for i, col in enumerate(players.columns)]
    team_name = fetch_team_name(team_url.split('/')[-1].replace('-Stats', '').replace('-', ' '))
    sql_team_id = "SELECT id FROM Team WHERE team_name = %s"
    cursor.execute(sql_team_id,(team_name,))
    team_id = cursor.fetchone()
    for index, row in players.iterrows():
        sql = """
        INSERT INTO Player (player_name, Team_id, country, position, age, matches_played, started, minutes, goals, assist, penaltys_made, penaltys_attempted, yellow_card, red_card)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        player_values = (
            row['Player'],  # Nome do jogador
            team_id[0],  # ID do time 
            translate_country(row['Nation'].split()[-1]),  # Traduzir nacionalidade
            translate_pos(row['Pos'].strip()),
            safe_int(row['Age'].split('-')[0]),  # Idade
            safe_int(row['MP']),  # Jogos disputados
            safe_int(row['Starts']),  # Começou jogos
            safe_int(row['Min']),  # Minutos jogados
            safe_int(row['Gls']),  # Gols
            safe_int(row['Ast']),  # Assistências
            safe_int(row['PK']) if 'PK' in row else 0,  # Penaltys convertidos em gol, 0 se não existir
            safe_int(row['PKatt']) if 'PKatt' in row else 0,  # Penaltys tentados, 0 se não existir
            safe_int(row['CrdY']) if 'CrdY' in row else 0,  # Cartões amarelos, 0 se não existir
            safe_int(row['CrdR']) if 'CrdR' in row else 0   # Cartões vermelhos, 0 se não existir
        )
        cursor.execute(sql, player_values)
    time.sleep(2)
conn.commit()
cursor.close()
closeConnection(conn)



C:\Users\Carlos Vinícius\AppData\Local\Temp\ipykernel_9480\1771754444.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  players = pd.read_html(data.text, match="Standard Stats")[0]


KeyboardInterrupt: 

In [25]:
#Extração de informação dos players dos times e atualizando os valores
conn = openConnection()
cursor = conn.cursor()
for team_url in team_urls:
    data = requests.get(team_url)
    players = pd.read_html(data.text, match="Standard Stats")[0]
    players.columns = players.columns.droplevel(0)
    players.columns = players.columns.get_level_values(-1) 
    players = players[:-2]
    players.columns = [f"{col}_{i}" if players.columns.duplicated()[i] else col for i, col in enumerate(players.columns)]
    team_name = fetch_team_name(team_url.split('/')[-1].replace('-Stats', '').replace('-', ' '))
    sql_team_id = "SELECT id FROM Team WHERE team_name = %s"
    cursor.execute(sql_team_id,(team_name,))
    team_id = cursor.fetchone()
    for index, row in players.iterrows():
        sql_update = """
            UPDATE Player 
            SET 
                country = %s, 
                position = %s, 
                age = %s, 
                matches_played = %s, 
                started = %s, 
                minutes = %s, 
                goals = %s, 
                assist = %s, 
                penaltys_made = %s, 
                penaltys_attempted = %s, 
                yellow_card = %s, 
                red_card = %s 
            WHERE player_name = %s
            """
        player_values = (
            translate_country(row['Nation'].split()[-1]),  # Traduzir nacionalidade
            translate_pos(row['Pos'].strip()),
            safe_int(row['Age'].split('-')[0]),  # Idade
            safe_int(row['MP']),  # Jogos disputados
            safe_int(row['Starts']),  # Começou jogos
            safe_int(row['Min']),  # Minutos jogados
            safe_int(row['Gls']),  # Gols
            safe_int(row['Ast']),  # Assistências
            safe_int(row['PK']) if 'PK' in row else 0,  # Penaltys convertidos em gol, 0 se não existir
            safe_int(row['PKatt']) if 'PKatt' in row else 0,  # Penaltys tentados, 0 se não existir
            safe_int(row['CrdY']) if 'CrdY' in row else 0,  # Cartões amarelos, 0 se não existir
            safe_int(row['CrdR']) if 'CrdR' in row else 0,   # Cartões vermelhos, 0 se não existir
            row['Player'],  # Nome do jogador
        )
        cursor.execute(sql_update, player_values)
    time.sleep(2)
conn.commit()
cursor.close()
closeConnection(conn)

C:\Users\Carlos Vinícius\AppData\Local\Temp\ipykernel_21500\966906929.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  players = pd.read_html(data.text, match="Standard Stats")[0]


KeyboardInterrupt: 

In [14]:
# Função tradução dos dias da semana
def translate_day(day_code):
    week_days = {
        'Mon': 'Segunda-feira',
        'Tue': 'Terça-feira',
        'Wed': 'Quarta-feira',
        'Thu': 'Quinta-feira',
        'Fri': 'Sexta-feira',
        'Sat': 'Sábado',
        'Sun': 'Domingo'
    } 
    return week_days.get(day_code, 'Sigla inválida')

In [15]:
#Função de tradução do resultado da partida
def translate_result(result):
    results = {
        'W': 'Vitória',
        'D': 'Empate',
        'L': 'Derrota'
    }
    return results.get(result, 'Sigla inválida')

In [46]:
#pegando informação das partidas dos times na premier league a primeira vez
conn = openConnection()
cursor = conn.cursor(buffered=True)
matches_array = []
for team_url in team_urls:
    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    # Filtra apenas as partidas da Premier League
    premier_league_matches = matches[matches['Comp'] == 'Premier League']
    # Itera sobre as partidas e adiciona ao array
    for index, row in premier_league_matches.iterrows():
        venue = row['Venue']
        opponent_name = fetch_team_name(row['Opponent'])
        captain_name = row['Captain']
        if venue == 'Home':
            venue = 'Casa'
        if venue == 'Away':
            venue = 'Fora'
        # Consulta SQL para buscar o ID do oponente
        sql_opponent_id = "SELECT id FROM team WHERE team_name = %s"
        cursor.execute(sql_opponent_id, (opponent_name,))
        opponent_id = cursor.fetchone()  # Retorna o ID do oponente
        sql_captain_id = "SELECT id FROM player WHERE player_name = %s"
        captain_id = None
        if(type(captain_name) == str):
            cursor.execute(sql_captain_id, (captain_name,))
            captain_id = cursor.fetchone()[0]
        week_day = None
        week_day = translate_day(row['Day'].strip())
        result = None
        if(type(row["Result"]) == str):
            result = translate_result(row['Result'].strip())
        goals_for = row['GF']
        if(pd.isna(row['GF'])):
            goals_for = None
        goals_against = row['GA']
        if(pd.isna(row['GA'])):
            goals_against = None
        poss = row['Poss']
        if(pd.isna(row['Poss'])):
            poss = None
        formation = None
        if(type(row['Formation']) == str):
            formation = row['Formation']
        opp_formation = None
        if(type(row['Opp Formation']) == str):
            opp_formation = row['Opp Formation']
        team_name = fetch_team_name(team_url.split('/')[-1].replace('-Stats', '').replace('-', ' '))
        sql_team_id = "SELECT id FROM Team WHERE team_name = %s"
        cursor.execute(sql_team_id,(team_name,))
        team_id = cursor.fetchone()
        if opponent_id:
            opponent_id = opponent_id[0] 
        match_info = (
            row['Date'],            # Data da partida
            row['Round'],           #Rodada
            week_day,               #dia da semana
            venue,                  #Local da partida (casa/fora)
            result,                 #Resultado da partida
            goals_for,              #Gols marcados
            goals_against,              #Gols sofridos
            opponent_id,            #Adversário
            poss,            #Posse de bola
            captain_id,               #Capitão do time
            formation,       #Formação do time
            opp_formation,    #Formação do time adversário
            team_id[0]
        )
        sql = """
        INSERT INTO Matches (date, round, week_day, venue, result, goals_for, goals_against, opponent_team_id, poss, captain_id, formation, opponent_formation, team_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql ,match_info)
        matches_array.append(match_info)
    time.sleep(5)
conn.commit()
cursor.close()
closeConnection(conn)
matches_array

C:\Users\Carlos Vinícius\AppData\Local\Temp\ipykernel_9480\779610535.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


Ipswich Town Matchweek 1 Ipswich Town
Brentford Matchweek 2 Brentford
Manchester Utd Matchweek 3 Manchester Utd
Nott'ham Forest Matchweek 4 Nott'ham Forest
Bournemouth Matchweek 5 Bournemouth
Wolves Matchweek 6 Wolves
Crystal Palace Matchweek 7 Crystal Palace
Chelsea Matchweek 8 Chelsea
Arsenal Matchweek 9 Arsenal
Brighton Matchweek 10 Brighton
Aston Villa Matchweek 11 Aston Villa
Southampton Matchweek 12 Southampton
Manchester City Matchweek 13 Manchester City
Newcastle Utd Matchweek 14 Newcastle Utd
Everton Matchweek 15 Everton
Fulham Matchweek 16 Fulham
Tottenham Matchweek 17 Tottenham
Leicester City Matchweek 18 Leicester City
West Ham Matchweek 19 West Ham
Manchester Utd Matchweek 20 Manchester Utd
Nott'ham Forest Matchweek 21 Nott'ham Forest
Brentford Matchweek 22 Brentford
Ipswich Town Matchweek 23 Ipswich Town
Bournemouth Matchweek 24 Bournemouth
Wolves Matchweek 25 Wolves
Manchester City Matchweek 26 Manchester City
Newcastle Utd Matchweek 27 Newcastle Utd
Southampton Matchwee

C:\Users\Carlos Vinícius\AppData\Local\Temp\ipykernel_9480\779610535.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


Chelsea Matchweek 1 Chelsea
Ipswich Town Matchweek 2 Ipswich Town
West Ham Matchweek 3 West Ham
Brentford Matchweek 4 Brentford
Arsenal Matchweek 5 Arsenal
Newcastle Utd Matchweek 6 Newcastle Utd
Fulham Matchweek 7 Fulham
Wolves Matchweek 8 Wolves
Southampton Matchweek 9 Southampton
Bournemouth Matchweek 10 Bournemouth
Brighton Matchweek 11 Brighton
Tottenham Matchweek 12 Tottenham
Liverpool Matchweek 13 Liverpool
Nott'ham Forest Matchweek 14 Nott'ham Forest
Crystal Palace Matchweek 15 Crystal Palace
Manchester Utd Matchweek 16 Manchester Utd
Aston Villa Matchweek 17 Aston Villa
Everton Matchweek 18 Everton
Leicester City Matchweek 19 Leicester City
West Ham Matchweek 20 West Ham
Brentford Matchweek 21 Brentford
Ipswich Town Matchweek 22 Ipswich Town
Chelsea Matchweek 23 Chelsea
Arsenal Matchweek 24 Arsenal
Newcastle Utd Matchweek 25 Newcastle Utd
Liverpool Matchweek 26 Liverpool
Tottenham Matchweek 27 Tottenham
Nott'ham Forest Matchweek 28 Nott'ham Forest
Brighton Matchweek 29 Brighto

C:\Users\Carlos Vinícius\AppData\Local\Temp\ipykernel_9480\779610535.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


...

C:\Users\Carlos Vinícius\AppData\Local\Temp\ipykernel_9480\779610535.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


Newcastle Utd Matchweek 1 Newcastle Utd
Nott'ham Forest Matchweek 2 Nott'ham Forest
Brentford Matchweek 3 Brentford
Manchester Utd Matchweek 4 Manchester Utd
Ipswich Town Matchweek 5 Ipswich Town
Bournemouth Matchweek 6 Bournemouth
Arsenal Matchweek 7 Arsenal
Leicester City Matchweek 8 Leicester City
Manchester City Matchweek 9 Manchester City
Everton Matchweek 10 Everton
Wolves Matchweek 11 Wolves
Liverpool Matchweek 12 Liverpool
Brighton Matchweek 13 Brighton
Chelsea Matchweek 14 Chelsea
Aston Villa Matchweek 15 Aston Villa
Tottenham Matchweek 16 Tottenham
Fulham Matchweek 17 Fulham
West Ham Matchweek 18 West Ham
Crystal Palace Matchweek 19 Crystal Palace
Brentford Matchweek 20 Brentford
Manchester Utd Matchweek 21 Manchester Utd
Nott'ham Forest Matchweek 22 Nott'ham Forest
Newcastle Utd Matchweek 23 Newcastle Utd
Ipswich Town Matchweek 24 Ipswich Town
Bournemouth Matchweek 25 Bournemouth
Brighton Matchweek 26 Brighton
Chelsea Matchweek 27 Chelsea
Liverpool Matchweek 28 Liverpool
Wol

C:\Users\Carlos Vinícius\AppData\Local\Temp\ipykernel_9480\779610535.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


Arsenal Matchweek 1 Arsenal
Chelsea Matchweek 2 Chelsea
Nott'ham Forest Matchweek 3 Nott'ham Forest
Newcastle Utd Matchweek 4 Newcastle Utd
Aston Villa Matchweek 5 Aston Villa
Liverpool Matchweek 6 Liverpool
Brentford Matchweek 7 Brentford
Manchester City Matchweek 8 Manchester City
Brighton Matchweek 9 Brighton
Crystal Palace Matchweek 10 Crystal Palace
Southampton Matchweek 11 Southampton
Fulham Matchweek 12 Fulham
Bournemouth Matchweek 13 Bournemouth
Everton Matchweek 14 Everton
West Ham Matchweek 15 West Ham
Ipswich Town Matchweek 16 Ipswich Town
Leicester City Matchweek 17 Leicester City
Manchester Utd Matchweek 18 Manchester Utd
Tottenham Matchweek 19 Tottenham
Nott'ham Forest Matchweek 20 Nott'ham Forest
Newcastle Utd Matchweek 21 Newcastle Utd
Chelsea Matchweek 22 Chelsea
Arsenal Matchweek 23 Arsenal
Aston Villa Matchweek 24 Aston Villa
Liverpool Matchweek 25 Liverpool
Bournemouth Matchweek 26 Bournemouth
Fulham Matchweek 27 Fulham
Everton Matchweek 28 Everton
Southampton Match

[('2024-08-17',
  'Matchweek 1',
  'Sábado',
  'Fora',
  'Vitória',
  2.0,
  0.0,
  17,
  62.0,
  1,
  '4-2-3-1',
  '4-2-3-1',
  1),
 ('2024-08-25',
  'Matchweek 2',
  'Domingo',
  'Casa',
  'Vitória',
  2.0,
  0.0,
  13,
  62.0,
  1,
  '4-2-3-1',
  '4-4-2',
  1),
 ('2024-09-01',
  'Matchweek 3',
  'Domingo',
  'Fora',
  'Vitória',
  3.0,
  0.0,
  12,
  47.0,
  1,
  '4-2-3-1',
  '4-2-3-1',
  1),
 ('2024-09-14',
  'Matchweek 4',
  'Sábado',
  'Casa',
  'Derrota',
  0.0,
  1.0,
  8,
  68.0,
  1,
  '4-2-3-1',
  '4-2-3-1',
  1),
 ('2024-09-21',
  'Matchweek 5',
  'Sábado',
  'Casa',
  'Vitória',
  3.0,
  0.0,
  11,
  58.0,
  1,
  '4-2-3-1',
  '4-2-3-1',
  1),
 ('2024-09-28',
  'Matchweek 6',
  'Sábado',
  'Fora',
  'Vitória',
  2.0,
  1.0,
  20,
  55.0,
  1,
  '4-2-3-1',
  '4-1-4-1',
  1),
 ('2024-10-05',
  'Matchweek 7',
  'Sábado',
  'Fora',
  'Vitória',
  1.0,
  0.0,
  18,
  68.0,
  1,
  '4-2-3-1',
  '3-4-3',
  1),
 ('2024-10-20',
  'Matchweek 8',
  'Domingo',
  'Casa',
  'Vitória',
  2

In [17]:
cursor.close()
closeConnection(conn)

In [48]:
#pegando dados de chutes
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.head()

C:\Users\Carlos\AppData\Local\Temp\ipykernel_11836\190749613.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


For Liverpool                                                             \
           Date   Time            Comp        Round  Day Venue Result   GF   
0    2023-08-13  16:30  Premier League  Matchweek 1  Sun  Away      D  1.0   
1    2023-08-19  15:00  Premier League  Matchweek 2  Sat  Home      W  3.0   
2    2023-08-27  16:30  Premier League  Matchweek 3  Sun  Away      W  2.0   
3    2023-09-03  14:00  Premier League  Matchweek 4  Sun  Home      W  3.0   
4    2023-09-16  12:30  Premier League  Matchweek 5  Sat  Away      W  3.0   

                       ... Standard               Expected                    \
    GA       Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  1.0        Chelsea  ...     17.8  0.0  0     0      1.3  1.3    0.10 -0.3   
1  1.0    Bournemouth  ...     16.8  1.0  0     1      3.0  2.1    0.09  0.0   
2  1.0  Newcastle Utd  ...     17.2  1.0  0     0      0.9  0.9    0.10  1.1   
3  0.0    Aston Villa  ...     14.7  0.0  0     0      2.5  2.5    0.15 -0.5   
4  1.0         Wolves  ...     15.8  0.0  0     0      2.5  2.5    0.16 -0.5   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0    -0.3        Match Report  
1     0.9        Match Report  
2     1.1        Match Report  
3    -0.5        Match Report  
4    -0.5        Match Report  

[5 rows x 26 columns]

#Tratando os dados

In [49]:
shooting.columns = shooting.columns.droplevel()

In [63]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,4-3-3,Anthony Taylor,Match Report,NaN,13,1,17.8,0.0,0,0
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,4-3-3,Thomas Bramall,Match Report,NaN,25,9,16.8,1.0,0,1
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,4-3-3,John Brooks,Match Report,NaN,9,4,17.2,1.0,0,0
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,4-3-3,Simon Hooper,Match Report,NaN,17,4,14.7,0.0,0,0
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,4-3-3,Michael Oliver,Match Report,NaN,16,5,15.8,0.0,0,0
5,2023-09-21,18:45,Europa Lg,Group stage,Thu,Away,W,3.0,1.0,at LASK,...,4-3-3,Marco Di Bello,Match Report,NaN,13,4,16.1,0.0,1,1
6,2023-09-24,14:00,Premier League,Matchweek 6,Sun,Home,W,3.0,1.0,West Ham,...,4-3-3,Chris Kavanagh,Match Report,NaN,21,6,19.3,2.0,1,1
7,2023-09-27,19:45,EFL Cup,Third round,Wed,Home,W,3.0,1.0,Leicester City,...,4-3-3,Tim Robinson,Match Report,NaN,29,10,NaN,NaN,0,0
8,2023-09-30,17:30,Premier League,Matchweek 7,Sat,Away,L,1.0,2.0,Tottenham,...,4-3-3,Simon Hooper,Match Report,NaN,12,4,14.9,0.0,0,0
9,2023-10-05,20:00,Europa Lg,Group stage,Thu,Home,W,2.0,0.0,be Union SG,...,4-3-3,Morten Krogh,Match Report,NaN,19,9,17.2,0.0,0,0


#Pegando dados de multiplos times e multiplos anos

In [ ]:
years = list(range(2024, 2021, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

for year in years:
    print(year)
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        print(matches)
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how="left")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(3)


In [3]:
#Retirando informação de um único ano campeonato brasileiro
year = 2024
all_matches = []
#standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
standings_url = "https://fbref.com/en/comps/24/Serie-A-Stats"
data = requests.get(standings_url)
time.sleep(5)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links] 
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    time.sleep(5)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links_shooting = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links_shooting[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    links_goalkeeping = [l for l in links if l and 'all_comps/keeper/' in l] 
    data = requests.get(f"https://fbref.com{links_goalkeeping[0]}")
    time.sleep(5)
    goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()
    links_passing = [l for l in links if l and 'all_comps/passing/' in l] 
    data = requests.get(f"https://fbref.com{links_passing[0]}")
    passing = pd.read_html(data.text, match="Passing")[0]
    passing.columns = passing.columns.droplevel()
    links_gca = [l for l in links if l and 'all_comps/gca/' in l] 
    data = requests.get(f"https://fbref.com{links_gca[0]}")
    time.sleep(5)
    goal_shot_creation = pd.read_html(data.text, match="Goal and Shot Creation")[0]
    goal_shot_creation.columns = goal_shot_creation.columns.droplevel()
    links_defense = [l for l in links if l and 'all_comps/defense/' in l] 
    data = requests.get(f"https://fbref.com{links_defense[0]}")
    time.sleep(5)
    defense = pd.read_html(data.text, match="Defensive Actions")[0]
    defense.columns = defense.columns.droplevel()
    defense = defense.rename(columns={'Sh': 'Shbk'})
    links_possession = [l for l in links if l and 'all_comps/possession/' in l] 
    data = requests.get(f"https://fbref.com{links_possession[0]}")
    time.sleep(5)
    possession = pd.read_html(data.text, match="Possession")[0]
    possession.columns = possession.columns.droplevel()
    links_miscellaneous = [l for l in links if l and 'all_comps/misc/' in l] 
    data = requests.get(f"https://fbref.com{links_miscellaneous[0]}")
    time.sleep(5)
    miscellaneous = pd.read_html(data.text, match="Miscellaneous")[0]
    miscellaneous.columns = miscellaneous.columns.droplevel()
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how="left").merge(goalkeeping[["Date","SoTA", "Save%", "PKsv"]], on="Date", how="left").merge(passing[["Date","Cmp%", "PPA", "CrsPA", "PrgP"]], on="Date", how="left").merge(goal_shot_creation[["Date","SCA", "GCA"]], on="Date", how="left").merge(defense[["Date", "TklW", "Tkl%", "Shbk", "Pass", "Int", "Tkl+Int", "Clr", "Err"]],on="Date", how="left").merge(possession[["Date", "Touches", "Def Pen", "Att Pen", "Carries", "CPA"]], on="Date", how="left").merge(miscellaneous[["Date", "CrdY", "CrdR", "Fls", "Fld", "Off", "Won%"]], on="Date", how="left")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "Série A"]
        
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_6392\1604732303.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_6392\1604732303.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_6392\1604732303.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_6392\1604732303.py:32: FutureWarning: Passing literal ht

In [ ]:
#Retirando informação de um único ano campeonato inglÊs
year = 2024
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
#standings_url = "https://fbref.com/en/comps/24/Serie-A-Stats"
data = requests.get(standings_url)
time.sleep(5)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links] 
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    time.sleep(5)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links_shooting = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links_shooting[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    links_goalkeeping = [l for l in links if l and 'all_comps/keeper/' in l] 
    data = requests.get(f"https://fbref.com{links_goalkeeping[0]}")
    time.sleep(5)
    goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()
    links_passing = [l for l in links if l and 'all_comps/passing/' in l] 
    data = requests.get(f"https://fbref.com{links_passing[0]}")
    passing = pd.read_html(data.text, match="Passing")[0]
    passing.columns = passing.columns.droplevel()
    links_gca = [l for l in links if l and 'all_comps/gca/' in l] 
    data = requests.get(f"https://fbref.com{links_gca[0]}")
    time.sleep(5)
    goal_shot_creation = pd.read_html(data.text, match="Goal and Shot Creation")[0]
    goal_shot_creation.columns = goal_shot_creation.columns.droplevel()
    links_defense = [l for l in links if l and 'all_comps/defense/' in l] 
    data = requests.get(f"https://fbref.com{links_defense[0]}")
    time.sleep(5)
    defense = pd.read_html(data.text, match="Defensive Actions")[0]
    defense.columns = defense.columns.droplevel()
    defense = defense.rename(columns={'Sh': 'Shbk'})
    links_possession = [l for l in links if l and 'all_comps/possession/' in l] 
    data = requests.get(f"https://fbref.com{links_possession[0]}")
    time.sleep(5)
    possession = pd.read_html(data.text, match="Possession")[0]
    possession.columns = possession.columns.droplevel()
    links_miscellaneous = [l for l in links if l and 'all_comps/misc/' in l] 
    data = requests.get(f"https://fbref.com{links_miscellaneous[0]}")
    time.sleep(5)
    miscellaneous = pd.read_html(data.text, match="Miscellaneous")[0]
    miscellaneous.columns = miscellaneous.columns.droplevel()
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how="left").merge(goalkeeping[["Date","SoTA", "Save%", "PKsv"]], on="Date", how="left").merge(passing[["Date","Cmp%", "PPA", "CrsPA", "PrgP"]], on="Date", how="left").merge(goal_shot_creation[["Date","SCA", "GCA"]], on="Date", how="left").merge(defense[["Date", "TklW", "Tkl%", "Shbk", "Pass", "Int", "Tkl+Int", "Clr", "Err"]],on="Date", how="left").merge(possession[["Date", "Touches", "Def Pen", "Att Pen", "Carries", "CPA"]], on="Date", how="left").merge(miscellaneous[["Date", "CrdY", "CrdR", "Fls", "Fld", "Off", "Won%"]], on="Date", how="left")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "Premier League"]
        
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)

In [4]:
len(all_matches)


20

In [2]:

#Retirando informação de um único ano campeonato espanhol
year = 2024
all_matches = []
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"
#standings_url = "https://fbref.com/en/comps/24/Serie-A-Stats"
data = requests.get(standings_url)
time.sleep(5)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links] 
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    time.sleep(5)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links_shooting = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links_shooting[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    links_goalkeeping = [l for l in links if l and 'all_comps/keeper/' in l] 
    data = requests.get(f"https://fbref.com{links_goalkeeping[0]}")
    time.sleep(5)
    goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()
    links_passing = [l for l in links if l and 'all_comps/passing/' in l] 
    data = requests.get(f"https://fbref.com{links_passing[0]}")
    passing = pd.read_html(data.text, match="Passing")[0]
    passing.columns = passing.columns.droplevel()
    links_gca = [l for l in links if l and 'all_comps/gca/' in l] 
    data = requests.get(f"https://fbref.com{links_gca[0]}")
    time.sleep(5)
    goal_shot_creation = pd.read_html(data.text, match="Goal and Shot Creation")[0]
    goal_shot_creation.columns = goal_shot_creation.columns.droplevel()
    links_defense = [l for l in links if l and 'all_comps/defense/' in l] 
    data = requests.get(f"https://fbref.com{links_defense[0]}")
    time.sleep(5)
    defense = pd.read_html(data.text, match="Defensive Actions")[0]
    defense.columns = defense.columns.droplevel()
    defense = defense.rename(columns={'Sh': 'Shbk'})
    links_possession = [l for l in links if l and 'all_comps/possession/' in l] 
    data = requests.get(f"https://fbref.com{links_possession[0]}")
    time.sleep(5)
    possession = pd.read_html(data.text, match="Possession")[0]
    possession.columns = possession.columns.droplevel()
    links_miscellaneous = [l for l in links if l and 'all_comps/misc/' in l] 
    data = requests.get(f"https://fbref.com{links_miscellaneous[0]}")
    time.sleep(5)
    miscellaneous = pd.read_html(data.text, match="Miscellaneous")[0]
    miscellaneous.columns = miscellaneous.columns.droplevel()
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how="left").merge(goalkeeping[["Date","SoTA", "Save%", "PKsv"]], on="Date", how="left").merge(passing[["Date","Cmp%", "PPA", "CrsPA", "PrgP"]], on="Date", how="left").merge(goal_shot_creation[["Date","SCA", "GCA"]], on="Date", how="left").merge(defense[["Date", "TklW", "Tkl%", "Shbk", "Pass", "Int", "Tkl+Int", "Clr", "Err"]],on="Date", how="left").merge(possession[["Date", "Touches", "Def Pen", "Att Pen", "Carries", "CPA"]], on="Date", how="left").merge(miscellaneous[["Date", "CrdY", "CrdR", "Fls", "Fld", "Off", "Won%"]], on="Date", how="left")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "La Liga"]
        
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\4056151824.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\4056151824.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\4056151824.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\4056151824.py:32: FutureWarning: Passing literal ht

In [7]:
#Retirando informação de um único ano campeonato italiano
year = 2024
all_matches = []
standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"
data = requests.get(standings_url)
time.sleep(5)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links] 
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    time.sleep(5)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links_shooting = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links_shooting[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    links_goalkeeping = [l for l in links if l and 'all_comps/keeper/' in l] 
    data = requests.get(f"https://fbref.com{links_goalkeeping[0]}")
    time.sleep(5)
    goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()
    links_passing = [l for l in links if l and 'all_comps/passing/' in l] 
    data = requests.get(f"https://fbref.com{links_passing[0]}")
    passing = pd.read_html(data.text, match="Passing")[0]
    passing.columns = passing.columns.droplevel()
    links_gca = [l for l in links if l and 'all_comps/gca/' in l] 
    data = requests.get(f"https://fbref.com{links_gca[0]}")
    time.sleep(5)
    goal_shot_creation = pd.read_html(data.text, match="Goal and Shot Creation")[0]
    goal_shot_creation.columns = goal_shot_creation.columns.droplevel()
    links_defense = [l for l in links if l and 'all_comps/defense/' in l] 
    data = requests.get(f"https://fbref.com{links_defense[0]}")
    time.sleep(5)
    defense = pd.read_html(data.text, match="Defensive Actions")[0]
    defense.columns = defense.columns.droplevel()
    defense = defense.rename(columns={'Sh': 'Shbk'})
    links_possession = [l for l in links if l and 'all_comps/possession/' in l] 
    data = requests.get(f"https://fbref.com{links_possession[0]}")
    time.sleep(5)
    possession = pd.read_html(data.text, match="Possession")[0]
    possession.columns = possession.columns.droplevel()
    links_miscellaneous = [l for l in links if l and 'all_comps/misc/' in l] 
    data = requests.get(f"https://fbref.com{links_miscellaneous[0]}")
    time.sleep(5)
    miscellaneous = pd.read_html(data.text, match="Miscellaneous")[0]
    miscellaneous.columns = miscellaneous.columns.droplevel()
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how="left").merge(goalkeeping[["Date","SoTA", "Save%", "PKsv"]], on="Date", how="left").merge(passing[["Date","Cmp%", "PPA", "CrsPA", "PrgP"]], on="Date", how="left").merge(goal_shot_creation[["Date","SCA", "GCA"]], on="Date", how="left").merge(defense[["Date", "TklW", "Tkl%", "Shbk", "Pass", "Int", "Tkl+Int", "Clr", "Err"]],on="Date", how="left").merge(possession[["Date", "Touches", "Def Pen", "Att Pen", "Carries", "CPA"]], on="Date", how="left").merge(miscellaneous[["Date", "CrdY", "CrdR", "Fls", "Fld", "Off", "Won%"]], on="Date", how="left")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "Serie A"]
        
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\1471529025.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\1471529025.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\1471529025.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\1471529025.py:31: FutureWarning: Passing literal ht

In [16]:
#Retirando informação de um único ano campeonato francês
year = 2024
all_matches = []
standings_url = "https://fbref.com/en/comps/13/Ligue-1-Stats"
data = requests.get(standings_url)
time.sleep(5)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links] 
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    time.sleep(5)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links_shooting = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links_shooting[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    links_goalkeeping = [l for l in links if l and 'all_comps/keeper/' in l] 
    data = requests.get(f"https://fbref.com{links_goalkeeping[0]}")
    time.sleep(5)
    goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()
    links_passing = [l for l in links if l and 'all_comps/passing/' in l] 
    data = requests.get(f"https://fbref.com{links_passing[0]}")
    passing = pd.read_html(data.text, match="Passing")[0]
    passing.columns = passing.columns.droplevel()
    links_gca = [l for l in links if l and 'all_comps/gca/' in l] 
    data = requests.get(f"https://fbref.com{links_gca[0]}")
    time.sleep(5)
    goal_shot_creation = pd.read_html(data.text, match="Goal and Shot Creation")[0]
    goal_shot_creation.columns = goal_shot_creation.columns.droplevel()
    links_defense = [l for l in links if l and 'all_comps/defense/' in l] 
    data = requests.get(f"https://fbref.com{links_defense[0]}")
    time.sleep(5)
    defense = pd.read_html(data.text, match="Defensive Actions")[0]
    defense.columns = defense.columns.droplevel()
    defense = defense.rename(columns={'Sh': 'Shbk'})
    links_possession = [l for l in links if l and 'all_comps/possession/' in l] 
    data = requests.get(f"https://fbref.com{links_possession[0]}")
    time.sleep(5)
    possession = pd.read_html(data.text, match="Possession")[0]
    possession.columns = possession.columns.droplevel()
    links_miscellaneous = [l for l in links if l and 'all_comps/misc/' in l] 
    data = requests.get(f"https://fbref.com{links_miscellaneous[0]}")
    time.sleep(5)
    miscellaneous = pd.read_html(data.text, match="Miscellaneous")[0]
    miscellaneous.columns = miscellaneous.columns.droplevel()
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how="left").merge(goalkeeping[["Date","SoTA", "Save%", "PKsv"]], on="Date", how="left").merge(passing[["Date","Cmp%", "PPA", "CrsPA", "PrgP"]], on="Date", how="left").merge(goal_shot_creation[["Date","SCA", "GCA"]], on="Date", how="left").merge(defense[["Date", "TklW", "Tkl%", "Shbk", "Pass", "Int", "Tkl+Int", "Clr", "Err"]],on="Date", how="left").merge(possession[["Date", "Touches", "Def Pen", "Att Pen", "Carries", "CPA"]], on="Date", how="left").merge(miscellaneous[["Date", "CrdY", "CrdR", "Fls", "Fld", "Off", "Won%"]], on="Date", how="left")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "Ligue 1"]
        
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\29657340.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\29657340.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\29657340.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\29657340.py:31: FutureWarning: Passing literal html to 'r

In [20]:
#Retirando informação de um único ano campeonato português
year = 2024
all_matches = []
standings_url = "https://fbref.com/en/comps/32/Primeira-Liga-Stats"
data = requests.get(standings_url)
time.sleep(5)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links] 
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    time.sleep(5)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links_shooting = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links_shooting[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    links_goalkeeping = [l for l in links if l and 'all_comps/keeper/' in l] 
    data = requests.get(f"https://fbref.com{links_goalkeeping[0]}")
    time.sleep(5)
    goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()
    links_passing = [l for l in links if l and 'all_comps/passing/' in l] 
    data = requests.get(f"https://fbref.com{links_passing[0]}")
    passing = pd.read_html(data.text, match="Passing")[0]
    passing.columns = passing.columns.droplevel()
    links_gca = [l for l in links if l and 'all_comps/gca/' in l] 
    data = requests.get(f"https://fbref.com{links_gca[0]}")
    time.sleep(5)
    goal_shot_creation = pd.read_html(data.text, match="Goal and Shot Creation")[0]
    goal_shot_creation.columns = goal_shot_creation.columns.droplevel()
    links_defense = [l for l in links if l and 'all_comps/defense/' in l] 
    data = requests.get(f"https://fbref.com{links_defense[0]}")
    time.sleep(5)
    defense = pd.read_html(data.text, match="Defensive Actions")[0]
    defense.columns = defense.columns.droplevel()
    defense = defense.rename(columns={'Sh': 'Shbk'})
    links_possession = [l for l in links if l and 'all_comps/possession/' in l] 
    data = requests.get(f"https://fbref.com{links_possession[0]}")
    time.sleep(5)
    possession = pd.read_html(data.text, match="Possession")[0]
    possession.columns = possession.columns.droplevel()
    links_miscellaneous = [l for l in links if l and 'all_comps/misc/' in l] 
    data = requests.get(f"https://fbref.com{links_miscellaneous[0]}")
    time.sleep(5)
    miscellaneous = pd.read_html(data.text, match="Miscellaneous")[0]
    miscellaneous.columns = miscellaneous.columns.droplevel()
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date", how="left").merge(goalkeeping[["Date","SoTA", "Save%", "PKsv"]], on="Date", how="left").merge(passing[["Date","Cmp%", "PPA", "CrsPA", "PrgP"]], on="Date", how="left").merge(goal_shot_creation[["Date","SCA", "GCA"]], on="Date", how="left").merge(defense[["Date", "TklW", "Tkl%", "Shbk", "Pass", "Int", "Tkl+Int", "Clr", "Err"]],on="Date", how="left").merge(possession[["Date", "Touches", "Def Pen", "Att Pen", "Carries", "CPA"]], on="Date", how="left").merge(miscellaneous[["Date", "CrdY", "CrdR", "Fls", "Fld", "Off", "Won%"]], on="Date", how="left")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "Primeira Liga"]
        
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\3400238594.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\3400238594.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\3400238594.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1976\3400238594.py:31: FutureWarning: Passing literal ht

In [21]:
len(all_matches)

18

In [22]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,carries,cpa,crdy,crdr,fls,fld,off,won%,season,team
0,2023-08-12,20:30,Primeira Liga,Matchweek 1,Sat,Home,W,3.0,2.0,Vizela,...,431.0,13.0,4.0,0.0,16.0,9.0,0.0,62.5,2024,Sporting CP
1,2023-08-18,20:15,Primeira Liga,Matchweek 2,Fri,Away,W,2.0,1.0,Casa Pia,...,387.0,5.0,1.0,0.0,11.0,14.0,0.0,50.0,2024,Sporting CP
2,2023-08-27,20:30,Primeira Liga,Matchweek 3,Sun,Home,W,1.0,0.0,Famalicão,...,492.0,7.0,3.0,0.0,17.0,13.0,2.0,46.7,2024,Sporting CP
3,2023-09-03,20:30,Primeira Liga,Matchweek 4,Sun,Away,D,1.0,1.0,Braga,...,332.0,7.0,5.0,0.0,10.0,17.0,4.0,37.9,2024,Sporting CP
4,2023-09-17,20:30,Primeira Liga,Matchweek 5,Sun,Home,W,3.0,0.0,Moreirense,...,532.0,6.0,3.0,0.0,10.0,16.0,1.0,62.1,2024,Sporting CP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,2024-04-21,NaN,Primeira Liga,Matchweek 30,Sun,Home,NaN,NaN,NaN,Estoril,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,Chaves
30,2024-04-28,NaN,Primeira Liga,Matchweek 31,Sun,Away,NaN,NaN,NaN,Casa Pia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,Chaves
31,2024-05-05,NaN,Primeira Liga,Matchweek 32,Sun,Home,NaN,NaN,NaN,Porto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,Chaves
32,2024-05-12,NaN,Primeira Liga,Matchweek 33,Sun,Home,NaN,NaN,NaN,Famalicão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,Chaves


In [23]:
match_df.to_csv('./data/portugues.csv', index=False)